## Import libary and data

In [1]:
import os
import math
import torch
import pandas as pd

from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import DataCollatorForLanguageModeling
from transformers import TrainingArguments, Trainer
from huggingface_hub import notebook_login
from datasets import Dataset

d:\Asus\Ungdung\Miniconda\workspace\envs\AIOEx\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DATASET_PATH = "../data/poem_final.csv"
df = pd.read_csv(DATASET_PATH)
df

,Unnamed: 0,title,content,source,url
0,0,“Cái làm ta hạnh phúc”,Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều...,"Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,1,“Chiều vừa xốp trên tay”,Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...,"Nguồn: Lâm Huy Nhuận, Chiều có thật (thơ), NXB...",https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
2,2,“Dưới giàn hoa thiên lý...”,Dưới giàn hoa thiên lý\nMột mình anh đang ngồi...,"Nguồn: Nguyễn Nhật Ánh, Mắt biếc, NXB Trẻ, 2004",https://www.thivien.net/Nguy%E1%BB%85n-Nh%E1%B...
3,3,"“Đến, nhiều nơi để đến”","Đến, nhiều nơi để đến\nVề, trở lại với mình\nC...","Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,4,“Đừng bao giờ dại dột”,Đừng bao giờ dại dột\nĐem chuyện riêng của mìn...,"Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
...,...,...,...,...,...
185,95,Ám ảnh sông xưa,"Ôi, con sóng chết khô,\nvật vờ trong bùn quánh...",NaN,https://www.thivien.net/%C4%90%E1%BB%97-Qu%E1%...
186,96,Áng dương không biết sầu,Áng dương không biết sầu\nNằm mãi ở trên cao\n...,"Nguồn: Lâu Văn Mua, Tôi bay vào mắt em (thơ), ...",https://www.thivien.net/L%C3%A2u-V%C4%83n-Mua/...
187,97,Anh,Cây bút gẫy trong tay\nCặn mực khô đáy lọ\nÁnh...,19-7-1973\n\n[Thông tin 2 nguồn tham khảo đã đ...,https://www.thivien.net/Xu%C3%A2n-Qu%E1%BB%B3n...
188,98,Anh biết,Không có anh để già\nLàm sao em được trẻ\nMuốn...,NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...


## Setup dataframe to datasets for hunggingface 

### Split part poem

In [11]:
def split_content(content : str):
    sample = []

    poem_parts = content.split("\n\n")
    for part in poem_parts:
        sentences = part.split("\n")
        if len(sentences) == 4:
            sample.append(sentences)

    return sample

df['part_content'] = df['content'].apply(lambda x : split_content(x))
df['part_content']

0      [[Cái làm ta hạnh phúc, Thực ra cũng chẳng nhi...
1      [[Chiều vừa xốp trên tay, Chợt nghe thoáng ong...
2      [[Dưới giàn hoa thiên lý, Một mình anh đang ng...
3      [[Đến, nhiều nơi để đến, Về, trở lại với mình,...
4      [[Đừng bao giờ dại dột, Đem chuyện riêng của m...
                             ...                        
185    [[Ôi, con sóng chết khô,, vật vờ trong bùn quá...
186    [[Áng dương không biết sầu, Nằm mãi ở trên cao...
187                                                   []
188    [[Không có anh để già, Làm sao em được trẻ, Mu...
189    [[Buồn, thì buồn thiệt đó, Em à, có biết không...
Name: part_content, Length: 190, dtype: object

### Explode content to new dataframe

In [16]:
df_exploded = df.explode("part_content")
df_exploded.reset_index(inplace=True, drop=True)
df_exploded

,Unnamed: 0,title,content,source,url,part_content
0,0,“Cái làm ta hạnh phúc”,Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều...,"Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...,"[Cái làm ta hạnh phúc, Thực ra cũng chẳng nhiề..."
1,0,“Cái làm ta hạnh phúc”,Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều...,"Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...,"[Rồi thêm chút công việc, Cho ta làm hàng ngày..."
2,1,“Chiều vừa xốp trên tay”,Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...,"Nguồn: Lâm Huy Nhuận, Chiều có thật (thơ), NXB...",https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...,"[Chiều vừa xốp trên tay, Chợt nghe thoáng ong ..."
3,1,“Chiều vừa xốp trên tay”,Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...,"Nguồn: Lâm Huy Nhuận, Chiều có thật (thơ), NXB...",https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...,"[Ớt đỏ sao cứ đỏ, Táo chín cho thật vàng, Em đ..."
4,2,“Dưới giàn hoa thiên lý...”,Dưới giàn hoa thiên lý\nMột mình anh đang ngồi...,"Nguồn: Nguyễn Nhật Ánh, Mắt biếc, NXB Trẻ, 2004",https://www.thivien.net/Nguy%E1%BB%85n-Nh%E1%B...,"[Dưới giàn hoa thiên lý, Một mình anh đang ngồ..."
...,...,...,...,...,...,...
481,98,Anh biết,Không có anh để già\nLàm sao em được trẻ\nMuốn...,NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...,"[Đất hiến dâng tuổi trẻ, Trời buông thả tuổi g..."
482,99,Anh bốn mùa lập đông,"Buồn, thì buồn thiệt đó\nEm à, có biết không\n...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...,"[Buồn, thì buồn thiệt đó, Em à, có biết không,..."
483,99,Anh bốn mùa lập đông,"Buồn, thì buồn thiệt đó\nEm à, có biết không\n...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...,"[Vâng, phiến buồn còn đó, Em nào biết phải khô..."
484,99,Anh bốn mùa lập đông,"Buồn, thì buồn thiệt đó\nEm à, có biết không\n...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...,"[Ừ, nỗi buồn vậy đó, Em đừng vội bâng khuâng, ..."


### Clear null

In [27]:
print("Tổng số hàng null", df_exploded['part_content'].isnull().sum())
df_exploded.dropna(inplace=True, subset="part_content")
print("Tổng số hàng null", df_exploded['part_content'].isnull().sum())

Tổng số hàng null 45
Tổng số hàng null 0


### Change list to string

In [29]:
df_exploded["part_content"] = df_exploded["part_content"].apply(lambda x: "\n".join(x))
df_exploded

,Unnamed: 0,title,content,source,url,part_content
0,0,“Cái làm ta hạnh phúc”,Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều...,"Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...,Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều...
1,0,“Cái làm ta hạnh phúc”,Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều...,"Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...,Rồi thêm chút công việc\nCho ta làm hàng ngày\...
2,1,“Chiều vừa xốp trên tay”,Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...,"Nguồn: Lâm Huy Nhuận, Chiều có thật (thơ), NXB...",https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...,Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...
3,1,“Chiều vừa xốp trên tay”,Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...,"Nguồn: Lâm Huy Nhuận, Chiều có thật (thơ), NXB...",https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...,Ớt đỏ sao cứ đỏ\nTáo chín cho thật vàng\nEm đẹ...
4,2,“Dưới giàn hoa thiên lý...”,Dưới giàn hoa thiên lý\nMột mình anh đang ngồi...,"Nguồn: Nguyễn Nhật Ánh, Mắt biếc, NXB Trẻ, 2004",https://www.thivien.net/Nguy%E1%BB%85n-Nh%E1%B...,Dưới giàn hoa thiên lý\nMột mình anh đang ngồi...
...,...,...,...,...,...,...
481,98,Anh biết,Không có anh để già\nLàm sao em được trẻ\nMuốn...,NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...,Đất hiến dâng tuổi trẻ\nTrời buông thả tuổi gi...
482,99,Anh bốn mùa lập đông,"Buồn, thì buồn thiệt đó\nEm à, có biết không\n...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...,"Buồn, thì buồn thiệt đó\nEm à, có biết không\n..."
483,99,Anh bốn mùa lập đông,"Buồn, thì buồn thiệt đó\nEm à, có biết không\n...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...,"Vâng, phiến buồn còn đó\nEm nào biết phải khôn..."
484,99,Anh bốn mùa lập đông,"Buồn, thì buồn thiệt đó\nEm à, có biết không\n...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...,"Ừ, nỗi buồn vậy đó\nEm đừng vội bâng khuâng\nV..."


### Dataloader

In [31]:
TEST_SIZE = 0.1
poem_dataset = Dataset.from_pandas(df_exploded)
poem_dataset = poem_dataset.train_test_split(test_size=TEST_SIZE)
poem_dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'title', 'content', 'source', 'url', 'part_content', '__index_level_0__'],
        num_rows: 396
    })
    test: Dataset({
        features: ['Unnamed: 0', 'title', 'content', 'source', 'url', 'part_content', '__index_level_0__'],
        num_rows: 45
    })
})

## Preprocessing